## 1 Creating persistent data to the notebook
Until now you have created ephemeral pods that once stopped will lose his data. We can change this creating a persistent volume and a persistent volume claim to use the node storage to store data and mount it inside the pod.

To better undestand the configuration of the persistent volume read the kubernetes [documentation](https://kubernetes.io/docs/concepts/storage/persistent-volumes/) about persistent volume and persistent volume claims.

The goal of this notebook is to create a persistent volume and its respective claim and use it to store the files used in the notebook. The tutoriall that supported this notebook is [here](https://kubernetes.io/docs/tasks/configure-pod-container/configure-persistent-volume-storage/).

First lets make the pykube,json an yaml importations and create the API object to communicate with de cluster API.

In [3]:
import pykube
import json
import yaml

api = pykube.HTTPClient(pykube.KubeConfig.from_file("k3s.yaml"))

### 1.1 Creating the persistent volume

Run the code below to create and send to the API the information to create a persistent volume and a persistent volume claim.

In [5]:
pv = {'apiVersion': 'v1', 'kind': 'PersistentVolume', 'metadata': {'name': 'volume', 'labels': {'type': 'local'}}, 'spec': {'storageClassName': 'default', 'capacity': {'storage': '2Gi'}, 'accessModes': ['ReadWriteOnce'], 'hostPath': {'path': '/mnt/data'}}}
pvc = {'apiVersion': 'v1', 'kind': 'PersistentVolumeClaim', 'metadata': {'name': 'claim'}, 'spec': {'storageClassName': 'default', 'accessModes': ['ReadWriteOnce'], 'resources': {'requests': {'storage': '1Gi'}}}}

pykube.PersistentVolume(api,pv).create()
pykube.PersistentVolumeClaim(api,pvc).create()

the objects tha you have created are equivalent to:
### persistent volume
```yaml
apiVersion: v1
kind: PersistentVolume
metadata:
  labels:
    type: local
  name: volume
spec:
  accessModes:
  - ReadWriteOnce
  capacity:
    storage: 2Gi
  hostPath:
    path: /mnt/data
  storageClassName: default
```
### persistent volume claim
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: claim
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: default
```

Note somethings in the files:
* As we do not specify a volume type it assumes the default type (filesystem)
* We created a "storageClassName" tha identifies the objects. It will be used by the cluster to bind they.
* The hostpath on the persistentVolume identifies the folder on computer that will be mounted on the pod.
* You can specify the amount of storage used by either the volume and the claim.

### 1.2 Updating the deployment to get the changes on
Now let's get an upgrade on the deployment that we are using to make it call the persistent volume claim tha we have just created. To do it, you have to:
1. get the old deployment object
2. create a new dictionary descring the new deployment
3. apply it using the patch function

In [14]:
#USE PYKUBE TO GET THE OLD DEPLOYMNENT OBJECT REFERENCE
old = pykube.Deployment.objects(api).get_by_name("notebook")
print(old)

notebook


In [11]:
#CREATE THE NEW OBJECT INCLUDING THE VOLUME CONFIGURATION
dep = {'apiVersion': 'apps/v1', 'kind': 'Deployment', 'metadata': {'name': 'notebook'}, 'spec': {'replicas': 1, 'selector': {'matchLabels': {'app': 'notebook'}}, 'template': {'metadata': {'labels': {'app': 'notebook'}}, 'spec': {'containers': [{'name': 'notebook', 'image': 'arkaisho/minimal-notebook', 'ports': [{'containerPort': 8888}], 'volumeMounts': [{'mountPath': '/home/arkaisho', 'name': 'volume'}]}], 'volumes': [{'name': 'volume', 'persistentVolumeClaim': {'claimName': 'claim'}}]}}}}
print(yaml.dump(dep))

apiVersion: apps/v1
kind: Deployment
metadata:
  name: notebook
spec:
  replicas: 1
  selector:
    matchLabels:
      app: notebook
  template:
    metadata:
      labels:
        app: notebook
    spec:
      containers:
      - image: arkaisho/minimal-notebook
        name: notebook
        ports:
        - containerPort: 8888
        volumeMounts:
        - mountPath: /home/arkaisho
          name: volume
      volumes:
      - name: volume
        persistentVolumeClaim:
          claimName: claim



In [15]:
#APPLY THE CHANGES USING THE PATCH FUNCTION
old.patch(dep)

## 2 Test your volume
Open your file browser on your host computer and put some files on the /mnt/data folder (like the k3s.yaml config file). Open your [notebook](http://localhost:8888) and see that the files are loaded.